# **Data Processing with LLM**

In this part, we are going to use LLM QWEN 2.5 on the Few-shot classified texts, for extracting edges or relational data from the texts. Our purpose is to extract companies accross many industries that focuses on AI technologies.

In [ ]:
!pip install ollama pandas networkx matplotlib tqdm datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 2s (227 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initializ

In [ ]:
import os
import threading
import subprocess

def start_ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=start_ollama)
ollama_thread.start()

In [ ]:
!ollama pull qwen2.5

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 2bada8a74506...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 2bada8a74506...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 2bada8a74506...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 2bada8a74506...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 2bada8a74506...   0% ▕▏    0 B/4.7 GB     

In [ ]:
import json
import ollama
import networkx as nx
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from datasets import load_dataset

In [ ]:
#Data we produced via few-shot.
df = pd.read_csv('/content/relevant_data (4) 1.csv')

In [ ]:
#Our intended prompt focusing on extracting relation between companies, their technologies and what industry they refer to.
SYSTEM_PROMPT = """Extract relationships between AI companies and the technologies they use within specific business sectors from the given text. Focus only on relationships where a company owns, develops, or implements a specific technology. If an individual text does not contain qualifying relationships, return a JSON object with an empty "edges" array for that text.

Provide output in this JSON format:
{
 "edges": [
   {"from": "Company Name", "to": "Technology Name", "type": "relationship_type", "industry_type": "Business Sector"}
 ]
}
The "type" field should be "owns", "develops", or "implements".
The "industry_type" field should categorize the business sector into one of these types:
1. Finance
2. E-commerce
3. Healthcare
4. Manufacturing
5. Construction
6. Garments
7. Retail
8. Education
9. Entertainment and Media
10. Automotive
11. Smart Home and IoT
12. Telecommunications
13. Transportation and Logistics
14. Cybersecurity
15. Recruitment and Human Resources (HR)
16. Energy and Utilities
17. Data Analytics and Business Intelligence
18. Software Development
19. Agriculture
20. Real Estate

Ensure that if no qualifying company-technology relationships are identified in a specific text, the output JSON object for that text has an "edges" array, even if empty. English output only.

Examples based on the input articles:
1. {"from": "Stripe", "to": "AI-driven fraud detection system", "type": "develops", "industry_type": "Finance"}
2. {"from": "Amazon", "to": "Recommendation engine", "type": "implements", "industry_type": "E-commerce"}
3. {"from": "Tesla", "to": "Autonomous driving technology", "type": "develops", "industry_type": "Automotive"}
4. {"from": "Zara", "to": "AI-powered supply chain optimization", "type": "implements", "industry_type": "Garments"}
5. If no relationships are found in a text, output: {"edges": []}
"""


In [ ]:
def extract_relationships(article):
    prompt = f"""
    Extract key relationships between companies and AI technologies from this text:
    Title: {article.get('title', 'untitled')}
    Text: {article['text']}
    Focus on relationships where a company owns, develops, or implements a specific technology.
    Categorize each technology according to the industry_type categories provided:
    """
    response = ollama.chat(
        model='qwen2.5',
        messages=[
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': prompt},
        ],
        format='json',
        options={"temperature":0.1}
    )
    return response['message']['content']

In [ ]:
# list to store results and edges
results = []
edges_list = []

# Processing each row by passing the row (dictionary)
for i, (_, row) in enumerate(tqdm(df.iterrows(), desc="Processing articles", total=df.shape[0])):
    try:
        article = {'title': row.get('title', 'Untitled'), 'text': row['text']}
        extracted_data = json.loads(extract_relationships(article))

        # Extracting edges from the response
        edges = extracted_data.get("edges", [])

        results.append({
            'title': article['title'],
            'extracted_data': extracted_data
        })

        # Appending edges to the list
        for edge in edges:
            edges_list.append(edge)


    except json.JSONDecodeError as e:
        print(f"Error processing row with title '{article['title']}': {str(e)}")
        results.append({
            'title': article['title'],
            'extracted_data': {"edges": []}
        })

# Saving results
with open('company_technology_relationships.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("Results saved to company_technology_relationships.json")

# Saving edges
with open('edges_list.json', 'w', encoding='utf-8') as f:
    json.dump(edges_list, f, ensure_ascii=False, indent=2)

print("Edges saved to edges_list.json")


Processing articles:   0%|          | 0/1223 [00:00<?, ?it/s]

Results saved to company_technology_relationships.json
Edges saved to edges_list.json
